In [ ]:
import numpy as np
import importlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from cp2k_spm_tools import cp2k_grid_orbitals, cp2k_ftsts, qe_utils

In [ ]:
lat_param = 4.30

wfn_file = "/home/kristjan/local_work/_proj_charged_7agnr/50uc/PROJ-RESTART.wfn"
xyz_file = "/home/kristjan/local_work/_proj_charged_7agnr/50uc/C700H208_pk170139.xyz"
cp2k_inp = "/home/kristjan/local_work/_proj_charged_7agnr/50uc/cp2k.inp"
basis_file = "/home/kristjan/local_work/_proj_charged_7agnr/50uc/BR"

In [ ]:
# define global energy limits (eV)
emin = -1.0
emax = 0.5

In [ ]:
cp2k_grid_orb = cp2k_grid_orbitals.Cp2kGridOrbitals()
cp2k_grid_orb.read_cp2k_input(cp2k_inp)
cp2k_grid_orb.read_xyz(xyz_file)
cp2k_grid_orb.ase_atoms.center()
cp2k_grid_orb.read_basis_functions(basis_file)
cp2k_grid_orb.load_restart_wfn_file(wfn_file, emin=emin, emax=emax)

In [ ]:
# define evaluation region (plane)

plane_h = 3.5 # ang

atoms_max_z = np.max(cp2k_grid_orb.ase_atoms.positions[:, 2]) # ang
plane_z = atoms_max_z+plane_h

eval_reg = [None, None, [plane_z, plane_z]]

In [ ]:
cp2k_grid_orb.calc_morbs_in_region(0.10,
                                x_eval_region = eval_reg[0],
                                y_eval_region = eval_reg[1],
                                z_eval_region = eval_reg[2],
                                reserve_extrap = 0.0,
                                pbc = (True, True, False),
                                eval_cutoff = 12.0)

# QE bands (optional)

In [ ]:
qe_bands_dir = None

qe_kpts = None
qe_bands = None
if qe_bands_dir is not None:
    if not qe_bands_dir.endswith('/'):
        qe_bands_dir += '/'
    qe_kpts, qe_bands, qe_fermi_en = .qe_utils.read_band_data(qe_bands_dir)
    # Shift QE bands such that VB onset is 0
    qe_homo = qe_utils.vb_onset(qe_bands[0], qe_fermi_en)
    qe_gap_middle = qe_utils.gap_middle(qe_bands[0], qe_fermi_en)
    qe_bands -= qe_gap_middle

# FT-STS

In [ ]:
de = 0.005
fwhm = 0.025

In [ ]:
ftsts = cp2k_ftsts.FTSTS(cp2k_grid_orb)
ftsts.project_orbitals_1d(gauss_pos=0.0, gauss_fwhm=3.0)
ftsts.take_fts(crop_padding=True, crop_edges=24.0, remove_row_avg=True, padding=3.0)
ftsts.make_ftldos(emin, emax, de, fwhm)

In [ ]:
imshow_kwargs = {'aspect': 'auto',
                 'origin': 'lower',
                 #'cmap': 'jet',
                 'cmap': 'gist_ncar',
                }

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(ftsts.ldos.T, extent=ftsts.ldos_extent, **imshow_kwargs)
plt.show()

In [ ]:
ftldos, extent = ftsts.get_ftldos_bz(4, lat_param)

plt.figure(figsize=(14, 12))
plt.imshow(ftldos.T, extent=extent, vmax=0.8*np.max(ftldos), **imshow_kwargs)

if qe_bands is not None:
    for qe_band in qe_bands[0]:
        plt.plot(2*qe_kpts[:, 0]*2*np.pi/lat_param, qe_band, '-', color='r', linewidth=2.0)
        
plt.ylim([extent[2], extent[3]])
plt.show()

# Plot individual orbitals

In [ ]:
# select orbitals wrt to HOMO
index_start = -5
index_end = 6

i_spin = 0

In [ ]:
for i_mo_wrt_homo in range(index_start, index_end+1):
    i_mo = cp2k_grid_orb.i_homo_glob[i_spin] + i_mo_wrt_homo
    print("HOMO%+d, E=%.3f eV" % (i_mo_wrt_homo, cp2k_grid_orb.morb_energies[i_spin][i_mo]))
    morb = (cp2k_grid_orb.morb_grids[i_spin][i_mo, :, :, 0]).astype(np.float64)
    morb_amax = np.max(np.abs(morb))
    fig = plt.figure(figsize=(30, 5))
    gs1 = gridspec.GridSpec(2, 2, wspace=0.0, hspace=0.0)
    gs1.update(left=0.0, right=0.6, wspace=0.0)
    ax00 = fig.add_subplot(gs1[0, 0])
    ax01 = fig.add_subplot(gs1[0, 1])
    ax10 = fig.add_subplot(gs1[1, 0])
    ax11 = fig.add_subplot(gs1[1, 1])
    
    ax00.imshow(morb.T, vmin=-morb_amax, vmax=morb_amax,origin='lower', cmap='seismic')
    ax00.set_xticks([])
    ax00.set_yticks([])
    
    ax10.imshow((morb**2).T,origin='lower', cmap='gist_heat')
    ax10.set_xticks([])
    ax10.set_yticks([])
    
    ax01.plot(ftsts.morbs_1d[i_spin][:, i_mo])
    ax01.set_xticks([])
    ax01.set_yticks([])
    
    bz_boundary = 10*np.pi/lat_param
    i_bzb = np.argmin(np.abs(ftsts.k_arr - bz_boundary))
    ax11.plot(ftsts.morb_fts[i_spin][:i_bzb, i_mo])
    ax11.set_xticks([])
    ax11.set_yticks([])
    
    plt.show()